In [31]:
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path
from load_data import ImageDataSet


from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.models import Model



In [24]:
# Create an instance of the class with the data directory
data_set = ImageDataSet("./../../data")
X_train = data_set.data_train
X_test = data_set.data_test
y_train = data_set.labels_train


print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}") # 262144 = 512*512 :)
print(f"X_test shape: {X_test.shape}")

X_train shape: (1000, 512, 512)
y_train shape: (1000, 262144)
X_test shape: (500, 512, 512)


In [25]:
#Coding the dice loss to use it as loss for the u-net
def dice_loss(y_true, y_pred):
    numerator = 2 * K.sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = K.sum(y_true + y_pred, axis=(1, 2, 3))
    return 1 - numerator / denominator

In [32]:
def unet(input_shape=(512, 512, 1), num_classes=2):
    inputs = Input(shape=input_shape)
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(num_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss=sparse_categorical_crossentropy, metrics=['accuracy'])

    return model



In [27]:
X_train_processed = X_train[:200] 
X_train_processed = np.expand_dims(X_train_processed, axis=-1)

pixel_values = y_train[:200].transpose()
y_train_array = pixel_values.values.reshape(((200, 512, 512, 1)))
# Create a new array with values of 1 where the original array is not 0
y_train_processed = np.where(y_train_array == 0, 0, 1)

print(X_train_processed.shape,
y_train_processed.shape)

(200, 512, 512, 1) (200, 512, 512, 1)


In [28]:
X_train_tensor = tf.convert_to_tensor(X_train_processed, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_processed, dtype=tf.float32)

print(X_train_tensor.shape,
y_train_tensor.shape)

# Check for missing or invalid values in X_train_tensor
if np.isnan(X_train_tensor).any() or np.isinf(X_train_tensor).any():
    # Handle missing or invalid values in X_train_tensor
    print("X_train_tensor contains missing or invalid values")
else:
    print("All good for X_train!")

# Check for missing or invalid values in y_train_tensor
if np.isnan(y_train_tensor).any() or np.isinf(y_train_tensor).any():
    # Handle missing or invalid values in y_train_tensor
    print("y_train_tensor contains missing or invalid values")
else:
    print("All good for y_train!")

(200, 512, 512, 1) (200, 512, 512, 1)
All good for X_train!
All good for y_train!


In [34]:
model = unet()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_46 (Conv2D)             (None, 512, 512, 64  640         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_47 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_46[0][0]']              
                                )                                                           

In [35]:
history = model.fit(X_train_tensor, y_train_tensor, batch_size=4, epochs=8, validation_split=0.1)

Epoch 1/8
 2/45 [>.............................] - ETA: 18:10 - loss: 58494.9609 - accuracy: 0.4712

In [ ]:
model.save_weights('first_try.h5')

In [ ]:
X_test_processed = np.expand_dims(X_test, axis=-1)
X_test_processed.shape

y_pred = model.predict(X_test)

(500, 512, 512, 1)

In [ ]:
y_pred_2d = y_pred.reshape((500, -1))

# Convert the 2D numpy array to a pandas DataFrame
df = pd.DataFrame(y_pred_2d)

# Save the DataFrame to a CSV file
df.to_csv("y_pred.csv", index=True, header=True)

In [ ]:
def plot_slice_seg(slice_image, seg):
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(slice_image, cmap="gray")
    axes[1].imshow(slice_image, cmap="gray")
    seg_masked = np.ma.masked_where(seg.reshape((512,512)) == 0, (seg.reshape((512,512))))
    axes[1].imshow(seg_masked, cmap="tab20")
    plt.axis("off")


plot_slice_seg(X_test[5], test.iloc[5].values.reshape((512,512)))